In [ ]:
!pip install opencv-python-headless
!pip install pytesseract
!pip install pandas
!pip install requests

In [ ]:
from google.colab import files
uploaded = files.upload()

# After uploading, load the dataset
import pandas as pd
import io

# Use the uploaded file name
train_df = pd.read_csv(io.BytesIO(uploaded['sub_train (4).csv']))

Saving sub_train.csv to sub_train (4).csv


In [ ]:
import requests
import os
from google.colab import files

output_dir = '/content/images'
os.makedirs(output_dir, exist_ok=True)  # Create the 'images' directory if it doesn't exist

# Step 4: Column name containing image URLs (replace 'image_link' with the correct column name)
url_column = 'image_link'

# Step 5: List to store the paths of the downloaded images
image_paths = []

# Step 6: Download images and save their paths
for index, row in train_df.iterrows():
    url = row[url_column]
    # Extract the image filename from the URL
    filename = os.path.join(output_dir, os.path.basename(url))

    try:
        # Download the image
        response = requests.get(url, stream=True)
        response.raise_for_status()  # Check for request errors

        # Save the image to the output directory
        with open(filename, 'wb') as file:
            for chunk in response.iter_content(chunk_size=8192):
                file.write(chunk)

        print(f"Downloaded {filename}")
        image_paths.append(filename)  # Store the local path of the downloaded image
    except Exception as e:
        print(f"Failed to download {url}. Error: {e}")
        image_paths.append('')  # Append an empty string if the download fails

# Step 7: Add a new column for the image paths to the DataFrame
train_df['image_path'] = image_paths

# Step 8: Save the updated DataFrame as a new CSV file
updated_csv_filename = 'sub_train_with_paths.csv'
train_df.to_csv(updated_csv_filename, index=False)

# Step 9: Download the updated CSV file to your local machine
files.download(updated_csv_filename)

print("All images have been downloaded and the CSV has been updated.")

Streaming output truncated to the last 5000 lines.
Downloaded /content/images/51UGu-oqM4L.jpg
Downloaded /content/images/814KMZ8wYjL.jpg
Downloaded /content/images/61A8qUtEhZL.jpg
Downloaded /content/images/71Ipvc3BK8L.jpg
Downloaded /content/images/81Apx8oXyVL.jpg
Downloaded /content/images/914if86WlmL.jpg
Downloaded /content/images/81Ow4zAbeTL.jpg
Downloaded /content/images/61xZ5-SW9SL.jpg
Downloaded /content/images/71Bdky2X5IL.jpg
Downloaded /content/images/91q5SkyTRYL.jpg
Downloaded /content/images/91YLkNxqVZL.jpg
Downloaded /content/images/61+lLfogy0L.jpg
Downloaded /content/images/51xMoFT3xCL.jpg
Downloaded /content/images/71VIAoN9yNL.jpg
Downloaded /content/images/81ZF5hDB8ML.jpg
Downloaded /content/images/91R+3Tr-fuS.jpg
Downloaded /content/images/61k6H-CWcQL.jpg
Downloaded /content/images/61rPniW0LtL.jpg
Downloaded /content/images/71PAgipDTPL.jpg
Downloaded /content/images/712L5tlgvNL.jpg
Downloaded /content/images/71M6NUuyGaL.jpg
Downloaded /content/images/41NLbHn272L.jpg
Dow

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

All images have been downloaded and the CSV has been updated.


In [2]:
!zip -r /content/images.zip /content/images


	zip warning: name not matched: /content/images

zip error: Nothing to do! (try: zip -r /content/images.zip . -i /content/images)


In [ ]:
from google.colab import files
files.download('/content/images.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!apt-get install tesseract-ocr -y

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 2s (2,986 kB/s)
Selecting previously unselected package tesseract-ocr-eng.
(Reading database ... 123597 files and directories currently installed.)
Preparing to unpack .../tesseract-ocr-

In [1]:
import pandas as pd
import os
import cv2
import pytesseract
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer


pytesseract.pytesseract.tesseract_cmd = r'/usr/bin/tesseract'


csv_file_path = '/content/sub_train_with_paths.csv'


df = pd.read_csv(csv_file_path)

# Function to process images for OCR
def process_image_for_ocr(image_path):
    try:
        # Load the image
        image = cv2.imread(image_path)

        # Convert to grayscale
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        # Apply binarization for better OCR results
        _, binary_image = cv2.threshold(gray_image, 128, 255, cv2.THRESH_BINARY)

        # Resize for consistency (optional)
        resized_image = cv2.resize(binary_image, (300, 300))

        return resized_image
    except Exception as e:
        print(f"Error processing image {image_path}: {e}")
        return None

# Function to extract text using OCR
def extract_text_from_image(image_path):
    processed_image = process_image_for_ocr(image_path)
    if processed_image is not None:
        extracted_text = pytesseract.image_to_string(processed_image)
        return extracted_text
    return ""

# Apply OCR on all images and match entity values
def match_entity_values(df):
    extracted_values = []

    for index, row in df.iterrows():
        image_path = row['image_path']  # Path of the image
        entity_name = row['entity_name']  # Entity name to look for in the image

        if os.path.exists(image_path):
            # Extract text using OCR
            ocr_text = extract_text_from_image(image_path)

            # Check if the extracted text contains the entity name
            if entity_name.lower() in ocr_text.lower():
                extracted_values.append(ocr_text)  # Store the OCR result
            else:
                extracted_values.append("")  # Empty if entity name is not found
        else:
            extracted_values.append("")  # Empty if image is not found

    # Add OCR extracted values to the DataFrame
    df['ocr_extracted_value'] = extracted_values
    return df

# Apply OCR and match entity values
df = match_entity_values(df)

# Save the DataFrame with OCR results
df.to_csv('/content/sub_train_with_ocr.csv', index=False)
print("OCR applied, and CSV updated with extracted text.")

# Next step: Preparing data and training the model

# Function to prepare data for training
def prepare_data_for_training(df):
    df_clean = df.dropna(subset=['ocr_extracted_value', 'entity_value'])  # Remove rows with missing OCR values or entity values
    X = df_clean['ocr_extracted_value'].values
    y = df_clean['entity_value'].values

    # Convert text data to feature vectors using CountVectorizer (this is simple character encoding; you can modify it)
    vectorizer = CountVectorizer()
    X_vectorized = vectorizer.fit_transform(X)

    return X_vectorized, y

# Function to train the model
def train_model(X, y):
    # Split the data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Initialize and train the Random Forest model
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)

    # Predict on the test set
    y_pred = model.predict(X_test)

    # Calculate accuracy
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Model accuracy: {accuracy * 100:.2f}%")

    return model

# Prepare the data
X_vectorized, y = prepare_data_for_training(df)

# Train the model
trained_model = train_model(X_vectorized, y)

print("Model training completed.")

ModuleNotFoundError: No module named 'pytesseract'